In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import imgaug as ia
import imgaug.augmenters as iaa
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
def preprocess_image(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [299, 299])
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    #image /= 255.0  # normalize to [0,1] range
    
    return image

def load_and_preprocess_image(path: str):
    image = preprocess_image(tf.io.read_file(path))
    return image

batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def prepare(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (load_and_preprocess_image(x), y), 
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Batch all datasets
    ds = ds.batch(batch_size)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [3]:
dataset_train = pd.read_csv('train.csv')
train_path_ds = tf.data.Dataset.from_tensor_slices(dataset_train.path.values.ravel())
train_labels_ds = tf.data.Dataset.from_tensor_slices(dataset_train.label.values.ravel())
train_img_ds = prepare(tf.data.Dataset.zip((train_path_ds, train_labels_ds)), shuffle=True, augment=True)

In [4]:
dataset_test = pd.read_csv('test.csv')
test_path_ds = tf.data.Dataset.from_tensor_slices(dataset_test.path.values.ravel())
test_labels_ds = tf.data.Dataset.from_tensor_slices(dataset_test.label.values.ravel())
test_img_ds = prepare(tf.data.Dataset.zip((test_path_ds, test_labels_ds)))

In [5]:
from tensorflow.keras.optimizers import Adam, RMSprop

from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dropout, Dense
import tensorflow.keras.backend as K

def load_model(num_classes):

    model = Sequential()
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())

    model.add(Dense(64))
    model.add(Activation("relu"))
    #model.add(BatchNormalization())
    model.add(Dropout(0.3))
 
    # softmax classifier
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    # initiate Adam optimizer
    opt = Adam(learning_rate=1e-4, decay=1e-3/32)
    
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  
    return model

In [7]:
model = load_model(num_classes=2)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                         factor=0.2,
                                                         patience=5, 
                                                         min_lr=1e-5)
with tf.device('/GPU:0'):
    H = model.fit(train_img_ds,
                epochs=10, 
                validation_data = test_img_ds,
                callbacks=[reduce_lr],verbose=1)

Epoch 1/10
4/4 [==============================] - 1s 131ms/step - loss: 1.5878 - accuracy: 0.5323 - val_loss: 1.6804 - val_accuracy: 0.5745
Epoch 2/10
4/4 [==============================] - 0s 96ms/step - loss: 1.2825 - accuracy: 0.5403 - val_loss: 0.8564 - val_accuracy: 0.4255
Epoch 3/10
4/4 [==============================] - 0s 111ms/step - loss: 0.8071 - accuracy: 0.4435 - val_loss: 0.7103 - val_accuracy: 0.5957
Epoch 4/10
4/4 [==============================] - 0s 93ms/step - loss: 0.7380 - accuracy: 0.5565 - val_loss: 0.7065 - val_accuracy: 0.5745
Epoch 5/10
4/4 [==============================] - 0s 99ms/step - loss: 0.7083 - accuracy: 0.5323 - val_loss: 0.7016 - val_accuracy: 0.5319
Epoch 6/10
4/4 [==============================] - 0s 100ms/step - loss: 0.7050 - accuracy: 0.4516 - val_loss: 0.6975 - val_accuracy: 0.4255
Epoch 7/10
4/4 [==============================] - 0s 112ms/step - loss: 0.6951 - accuracy: 0.3790 - val_loss: 0.6947 - val_accuracy: 0.4255
Epoch 8/10
4/4 [=======

In [8]:
y_pred = np.argmax(model.predict(test_img_ds), axis=-1)
y_true = dataset_test.label.values.ravel()
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.43      1.00      0.60        20
           1       0.00      0.00      0.00        27

    accuracy                           0.43        47
   macro avg       0.21      0.50      0.30        47
weighted avg       0.18      0.43      0.25        47



C:\Users\andro\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
def inception(num_classes):
    #inp = tf.keras.layers.Input([None, None, 3], dtype = tf.float32)
    #inp = tf.keras.applications.inception_v3.preprocess_input(inp)
    core = tf.keras.applications.InceptionV3(include_top=False,
                                             classes=num_classes)
    for layer in core.layers[:-11]:
        layer.trainable=False
    inp = core.input
    x = core.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes)(x)
    x = Activation("softmax")(x)
    model = tf.keras.Model(inputs=[inp], outputs=[x])
    opt = Adam(learning_rate=5e-4, decay=1e-3/32)
    
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

In [17]:
model = inception(num_classes=2)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_190 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_190[0][0]                 
__________________________________________________________________________________________________
activation_196 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [18]:
with tf.device('/GPU:0'):
    H = model.fit(train_img_ds,
                    epochs=5, 
                    validation_data = test_img_ds,
                    callbacks=[reduce_lr],verbose=1)

Epoch 1/5
4/4 [==============================] - 3s 845ms/step - loss: 0.7755 - accuracy: 0.4113 - val_loss: 0.7420 - val_accuracy: 0.4255
Epoch 2/5
4/4 [==============================] - 1s 153ms/step - loss: 0.7386 - accuracy: 0.4919 - val_loss: 0.7179 - val_accuracy: 0.7234
Epoch 3/5
4/4 [==============================] - 1s 171ms/step - loss: 0.7435 - accuracy: 0.5565 - val_loss: 0.7244 - val_accuracy: 0.5745
Epoch 4/5
4/4 [==============================] - 1s 165ms/step - loss: 0.7293 - accuracy: 0.3790 - val_loss: 0.7213 - val_accuracy: 0.5957
Epoch 5/5
4/4 [==============================] - 1s 140ms/step - loss: 0.7196 - accuracy: 0.5323 - val_loss: 0.7141 - val_accuracy: 0.7021


In [19]:
y_pred = np.argmax(model.predict(test_img_ds), axis=-1)
y_true = dataset_test.label.values.ravel()
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65        20
           1       0.74      0.74      0.74        27

    accuracy                           0.70        47
   macro avg       0.70      0.70      0.70        47
weighted avg       0.70      0.70      0.70        47



In [25]:
def vgg16(num_classes):
    #inp = tf.keras.layers.Input([None, None, 3], dtype = tf.float32)
    #inp = tf.keras.applications.inception_v3.preprocess_input(inp)
    core = tf.keras.applications.VGG16(include_top=False,
                                             classes=num_classes)
    for layer in core.layers[:-3]:
        layer.trainable=False
    inp = core.input
    x = core.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes)(x)
    x = Activation("softmax")(x)
    model = tf.keras.Model(inputs=[inp], outputs=[x])
    opt = Adam(learning_rate=1e-4, decay=1e-3/32)
    
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

In [26]:
model = vgg16(num_classes=2)
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128) 

In [ ]:
with tf.device('/GPU:0'):
    H = model.fit(train_img_ds,
                    epochs=5, 
                    validation_data = test_img_ds,
                    callbacks=[reduce_lr],verbose=1)

Epoch 1/5
4/4 [==============================] - ETA: 0s - loss: 0.7962 - accuracy: 0.4919

In [ ]:
y_pred = np.argmax(model.predict(test_img_ds), axis=-1)
y_true = dataset_test.label.values.ravel()
print(classification_report(y_true,y_pred))

In [109]:
def resnet50(num_classes):
    #inp = tf.keras.layers.Input([None, None, 3], dtype = tf.float32)
    #inp = tf.keras.applications.inception_v3.preprocess_input(inp)
    core = tf.keras.applications.ResNet50V2(include_top=False,
                                             classes=num_classes)
    for layer in core.layers[:-1]:
        layer.trainable=False
    inp = core.input
    x = core.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes)(x)
    x = Activation("softmax")(x)
    model = tf.keras.Model(inputs=[inp], outputs=[x])
    opt = Adam(learning_rate=5e-4, decay=1e-3/32)
    
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
    return model

In [110]:
model = resnet50(num_classes=2)
model.summary()

Model: "functional_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_24[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, None, None, 6 0           conv1_conv[0][0]                 
______________________________________________________________________________________

In [113]:
H = model.fit(train_img_ds,
                epochs=5, 
                validation_data = test_img_ds,
                callbacks=[reduce_lr],verbose=1)

Epoch 1/5
4/4 [==============================] - 1s 190ms/step - loss: 0.7258 - accuracy: 0.5161 - val_loss: 0.7239 - val_accuracy: 0.4043
Epoch 2/5
4/4 [==============================] - 1s 170ms/step - loss: 0.7235 - accuracy: 0.5403 - val_loss: 0.7214 - val_accuracy: 0.4468
Epoch 3/5
4/4 [==============================] - 1s 197ms/step - loss: 0.7304 - accuracy: 0.4758 - val_loss: 0.7274 - val_accuracy: 0.3830
Epoch 4/5
4/4 [==============================] - 1s 190ms/step - loss: 0.7206 - accuracy: 0.4435 - val_loss: 0.7217 - val_accuracy: 0.4255
Epoch 5/5
4/4 [==============================] - 1s 168ms/step - loss: 0.7293 - accuracy: 0.5565 - val_loss: 0.7180 - val_accuracy: 0.4043


In [114]:
y_pred = np.argmax(model.predict(test_img_ds), axis=-1)
y_true = dataset_test.label.values.ravel()
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.37      0.55      0.44        20
           1       0.47      0.30      0.36        27

    accuracy                           0.40        47
   macro avg       0.42      0.42      0.40        47
weighted avg       0.43      0.40      0.40        47

